In [1]:
import pandas as pd
import numpy as np

import networkx as nx

In [2]:
COUNTRIES = [
    {'id': '660', 'text': 'Anguilla'}, {'id': '28', 'text': 'Antigua and Barbuda'}, {'id': '32', 'text': 'Argentina'}, {'id': '533', 'text': 'Aruba'},
    {'id': '44', 'text': 'Bahamas'}, {'id': '52', 'text': 'Barbados'}, {'id': '84', 'text': 'Belize'}, {'id': '60', 'text': 'Bermuda'},
    {'id': '68', 'text': 'Bolivia (Plurinational State of)'}, {'id': '535', 'text': 'Bonaire'}, {'id': '76', 'text': 'Brazil'},
    {'id': '92', 'text': 'Br. Virgin Isds'}, {'id': '124', 'text': 'Canada'}, {'id': '136', 'text': 'Cayman Isds'}, {'id': '152', 'text': 'Chile'},
    {'id': '170', 'text': 'Colombia'}, {'id': '188', 'text': 'Costa Rica'}, {'id': '192', 'text': 'Cuba'},
    {'id': '531', 'text': 'Curaçao'}, {'id': '212', 'text': 'Dominica'}, {'id': '214', 'text': 'Dominican Rep.'}, {'id': '218', 'text': 'Ecuador'},
    {'id': '222', 'text': 'El Salvador'}, {'id': '238', 'text': 'Falkland Isds (Malvinas)'}, {'id': '254', 'text': 'French Guiana'}, {'id': '304', 'text': 'Greenland'},
    {'id': '308', 'text': 'Grenada'}, {'id': '312', 'text': 'Guadeloupe'}, {'id': '320', 'text': 'Guatemala'}, {'id': '328', 'text': 'Guyana'},
    {'id': '332', 'text': 'Haiti'}, {'id': '340', 'text': 'Honduras'}, {'id': '388', 'text': 'Jamaica'}, {'id': '474', 'text': 'Martinique'},
    {'id': '484', 'text': 'Mexico'}, {'id': '500', 'text': 'Montserrat'}, {'id': '558', 'text': 'Nicaragua'}, {'id': '591', 'text': 'Panama'},
    {'id': '600', 'text': 'Paraguay'}, {'id': '604', 'text': 'Peru'}, {'id': '461', 'text': 'Sabah'}, {'id': '652', 'text': 'Saint Barthelemy'},
    {'id': '659', 'text': 'Saint Kitts and Nevis'}, {'id': '658', 'text': 'Saint Kitts, Nevis and Anguilla'}, {'id': '662', 'text': 'Saint Lucia'},
    {'id': '534', 'text': 'Saint Maarten'}, {'id': '666', 'text': 'Saint Pierre and Miquelon'}, {'id': '670', 'text': 'Saint Vincent and the Grenadines'},
    {'id': '740', 'text': 'Suriname'}, {'id': '780', 'text': 'Trinidad and Tobago'}, {'id': '796', 'text': 'Turks and Caicos Isds'},
    {'id': '858', 'text': 'Uruguay'}, {'id': '850', 'text': 'US Virgin Isds'}, {'id': '842', 'text': 'USA'}, {'id': '841', 'text': 'USA (before 1981)'},
    {'id': '862', 'text': 'Venezuela'},
]

In [3]:
COUNTRIES_ID = set(int(c['id']) for c in COUNTRIES)

### OIL DATA

In [49]:
oil_df = pd.read_csv('data/americas-oil-trade-2011-2021.csv', dtype={
    'Year': 'int', 'Reporter Code': 'int', 'Partner Code': 'int', 'Commodity Code': 'int'
})

In [50]:
oil_df.head()

,Year,Trade Flow,Reporter Code,Reporter,Reporter ISO,Partner Code,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit,Qty,Trade Value (US$)
0,2011,Import,28,Antigua and Barbuda,ATG,0,World,WLD,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.500000e+01,5.230000e+02
1,2011,Import,28,Antigua and Barbuda,ATG,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.500000e+01,5.230000e+02
2,2011,Import,32,Argentina,ARG,0,World,WLD,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,8.040000e+02,5.050000e+03
3,2011,Export,32,Argentina,ARG,0,World,WLD,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,3.023702e+09,2.182224e+09
4,2011,Import,32,Argentina,ARG,24,Angola,AGO,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,3.500000e+01,5.950000e+02


In [172]:
country_codes = pd.concat([
    oil_df[['Reporter Code', 'Reporter', 'Reporter ISO']].rename(columns={'Reporter Code':'Code', 'Reporter':'Name', 'Reporter ISO': 'ISO'}),
    oil_df[['Partner Code', 'Partner', 'Partner ISO']].rename(columns={'Partner Code':'Code', 'Partner':'Name', 'Partner ISO': 'ISO'}),
]).groupby('Code').agg({
    'Name': 'last',
    'ISO': 'last'
}).reset_index()

In [174]:
country_codes.loc[:, 'Americas'] = country_codes.Code.apply(lambda x: x in COUNTRIES_ID)

In [175]:
country_codes.to_dict(orient='records')

[{'Code': 8, 'Name': 'Albania', 'ISO': 'ALB', 'Americas': False},
 {'Code': 12, 'Name': 'Algeria', 'ISO': 'DZA', 'Americas': False},
 {'Code': 24, 'Name': 'Angola', 'ISO': 'AGO', 'Americas': False},
 {'Code': 28, 'Name': 'Antigua and Barbuda', 'ISO': 'ATG', 'Americas': True},
 {'Code': 31, 'Name': 'Azerbaijan', 'ISO': 'AZE', 'Americas': False},
 {'Code': 32, 'Name': 'Argentina', 'ISO': 'ARG', 'Americas': True},
 {'Code': 36, 'Name': 'Australia', 'ISO': 'AUS', 'Americas': False},
 {'Code': 40, 'Name': 'Austria', 'ISO': 'AUT', 'Americas': False},
 {'Code': 44, 'Name': 'Bahamas', 'ISO': 'BHS', 'Americas': True},
 {'Code': 52, 'Name': 'Barbados', 'ISO': 'BRB', 'Americas': True},
 {'Code': 56, 'Name': 'Belgium', 'ISO': 'BEL', 'Americas': False},
 {'Code': 60, 'Name': 'Bermuda', 'ISO': 'BMU', 'Americas': True},
 {'Code': 68,
  'Name': 'Bolivia (Plurinational State of)',
  'ISO': 'BOL',
  'Americas': True},
 {'Code': 76, 'Name': 'Brazil', 'ISO': 'BRA', 'Americas': True},
 {'Code': 84, 'Name':

#### EXCLUDE AGGREGATES

In [53]:
exclude_codes = [899, 838, 837, 637, 568, 490, 473, 0]

oil_df = oil_df.loc[
    ~oil_df['Partner Code'].isin(exclude_codes)
]

### ADD EDGES/CONNECTIONS

In [54]:
def add_edge(row):
    #edge = frozenset([row['Year'], row['Partner ISO'], row['Reporter ISO']])
    #edge = frozenset([row['Partner ISO'], row['Reporter ISO']])
    edge = frozenset([
        row['Partner ISO'],
        row['Reporter ISO'],
        #row['Trade Flow']
    ])
    return edge

def edge(*args):
    return frozenset(args)

In [55]:
oil_df.loc[:, 'Edge'] = oil_df.apply(add_edge, axis=1)

In [56]:
oil_df.head()

,Year,Trade Flow,Reporter Code,Reporter,Reporter ISO,Partner Code,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit,Qty,Trade Value (US$),Edge
1,2011,Import,28,Antigua and Barbuda,ATG,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.500000e+01,523.0,"(ATG, USA)"
4,2011,Import,32,Argentina,ARG,24,Angola,AGO,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,3.500000e+01,595.0,"(AGO, ARG)"
5,2011,Import,32,Argentina,ARG,76,Brazil,BRA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.600000e+01,1266.0,"(BRA, ARG)"
6,2011,Export,32,Argentina,ARG,76,Brazil,BRA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,2.090765e+08,169565732.0,"(BRA, ARG)"
7,2011,Export,32,Argentina,ARG,152,Chile,CHL,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.029503e+09,717117739.0,"(CHL, ARG)"


In [72]:
oil_df.loc[
    oil_df['Edge'] == edge('CAN', 'BRA')
]

,Year,Trade Flow,Reporter Code,Reporter,Reporter ISO,Partner Code,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit,Qty,Trade Value (US$),Edge
37,2011,Import,76,Brazil,BRA,124,Canada,CAN,270900,"Petroleum oils, oils from bituminous minerals,...",No Quantity,0.000000e+00,4255.0,"(BRA, CAN)"
38,2011,Export,76,Brazil,BRA,124,Canada,CAN,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,9.668479e+08,634277847.0,"(BRA, CAN)"
75,2011,Import,124,Canada,CAN,76,Brazil,BRA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.099840e+09,633486028.0,"(BRA, CAN)"
382,2012,Export,76,Brazil,BRA,124,Canada,CAN,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,9.815899e+08,714429880.0,"(BRA, CAN)"
420,2012,Import,124,Canada,CAN,76,Brazil,BRA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,9.283782e+08,776451113.0,"(BRA, CAN)"
854,2013,Export,76,Brazil,BRA,124,Canada,CAN,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,7.945070e+08,560281604.0,"(BRA, CAN)"
887,2013,Import,124,Canada,CAN,76,Brazil,BRA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,8.140220e+08,680809052.0,"(BRA, CAN)"
954,2014,Export,76,Brazil,BRA,124,Canada,CAN,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,2.184347e+08,131552983.0,"(BRA, CAN)"
997,2014,Import,124,Canada,CAN,76,Brazil,BRA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,2.786603e+08,228331711.0,"(BRA, CAN)"
1063,2015,Export,76,Brazil,BRA,124,Canada,CAN,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,2.733720e+08,99334291.0,"(BRA, CAN)"


### GROUP BY YEAR + EDGE AND NORMALIZE EXPORT/IMPORT VALUES

In [153]:
def get_import(tx):
    return tx.loc[ tx['Trade Flow'] == 'Import', 'Trade Value (US$)'].sum()

def get_export(tx):
    #return tx.loc[ tx['Trade Flow'] == 'Export', 'Trade Value (US$)'].sum()
    val = oil_df.loc[
        (oil_df.Year == tx.Year.values[0]) & \
        (oil_df['Partner ISO'] == tx['Reporter ISO'].values[0]) & \
        (oil_df['Reporter ISO'] == tx['Partner ISO'].values[0]) & \
        (oil_df['Trade Flow'] == 'Import'),
        'Trade Value (US$)'
    ]
    
    if len(val) > 0:
        return val.iloc[0]
    else:
        return tx.loc[ tx['Trade Flow'] == 'Export', 'Trade Value (US$)'].sum()

In [161]:
edges_grp = oil_df.groupby(['Edge', 'Year', 'Reporter ISO', 'Partner ISO'])
edges = pd.DataFrame(index=edges_grp.groups.keys())

In [162]:
edges.loc[:, 'Trade Value Import'] = edges_grp.apply(get_import)

In [163]:
edges.loc[:, 'Trade Value Export'] = edges_grp.apply(get_export)

In [164]:
edges = edges.reset_index().rename(columns={
    'level_0': 'Edge',
    'level_1': 'Year',
    'level_2': 'Reporter ISO',
    'level_3': 'Partner ISO'
})

In [165]:
edges.loc[
    edges.Edge == edge('CAN', 'BRA')
].sort_values(by='Year')

,Edge,Year,Reporter ISO,Partner ISO,Trade Value Import,Trade Value Export
1837,"(BRA, CAN)",2011,BRA,CAN,4255.0,633486028.0
1893,"(BRA, CAN)",2011,CAN,BRA,633486028.0,4255.0
1942,"(BRA, CAN)",2012,BRA,CAN,0.0,776451113.0
1971,"(BRA, CAN)",2012,CAN,BRA,776451113.0,0.0
1561,"(BRA, CAN)",2013,CAN,BRA,680809052.0,0.0
1704,"(BRA, CAN)",2013,BRA,CAN,0.0,680809052.0
1617,"(BRA, CAN)",2014,BRA,CAN,0.0,228331711.0
2523,"(BRA, CAN)",2014,CAN,BRA,228331711.0,0.0
2429,"(BRA, CAN)",2015,CAN,BRA,143237533.0,0.0
2575,"(BRA, CAN)",2015,BRA,CAN,0.0,143237533.0


In [166]:
edges.loc[
    edges.Edge == edge('USA', 'VEN')
].sort_values(by='Year')

,Edge,Year,Reporter ISO,Partner ISO,Trade Value Import,Trade Value Export
2049,"(USA, VEN)",2011,USA,VEN,3.791359e+10,6.380000e+02
2052,"(USA, VEN)",2011,VEN,USA,6.380000e+02,3.791359e+10
1431,"(USA, VEN)",2012,USA,VEN,3.464306e+10,3.070000e+02
1432,"(USA, VEN)",2012,VEN,USA,3.070000e+02,3.464306e+10
609,"(USA, VEN)",2013,USA,VEN,2.815425e+10,0.000000e+00
662,"(USA, VEN)",2014,USA,VEN,2.641598e+10,0.000000e+00
535,"(USA, VEN)",2015,USA,VEN,1.374022e+10,0.000000e+00
123,"(USA, VEN)",2016,USA,VEN,9.767841e+09,0.000000e+00
23,"(USA, VEN)",2017,USA,VEN,1.071999e+10,0.000000e+00
760,"(USA, VEN)",2018,USA,VEN,1.062935e+10,0.000000e+00


In [167]:
edges.loc[
    edges.Edge == edge('USA', 'CAN')
].sort_values(by='Year')

,Edge,Year,Reporter ISO,Partner ISO,Trade Value Import,Trade Value Export
2009,"(USA, CAN)",2011,USA,CAN,6.914651e+10,1.409298e+09
1879,"(USA, CAN)",2011,CAN,USA,1.409298e+09,6.914651e+10
1962,"(USA, CAN)",2012,CAN,USA,2.332655e+09,7.421417e+10
1400,"(USA, CAN)",2012,USA,CAN,7.421417e+10,2.332655e+09
405,"(USA, CAN)",2013,USA,CAN,7.848179e+10,4.903731e+09
1609,"(USA, CAN)",2013,CAN,USA,4.903731e+09,7.848179e+10
2559,"(USA, CAN)",2014,CAN,USA,1.128628e+10,8.564660e+10
647,"(USA, CAN)",2014,USA,CAN,8.564660e+10,1.128628e+10
2502,"(USA, CAN)",2015,CAN,USA,7.698895e+09,4.988465e+10
545,"(USA, CAN)",2015,USA,CAN,4.988465e+10,7.698895e+09


### SUMMARY

In [178]:
edges.loc[
    edges['Reporter ISO'].isin(country_codes.loc[country_codes.Americas == True, 'ISO'])
].groupby('Reporter ISO').agg({
    'Trade Value Import': 'sum',
    'Trade Value Export': 'sum'
}).sort_values(by='Trade Value Export')

,Trade Value Import,Trade Value Export
Reporter ISO,,
PRY,4.466300e+05,0.000000e+00
ATG,3.868000e+03,0.000000e+00
MSR,7.771700e+04,0.000000e+00
VCT,7.480000e+02,0.000000e+00
BMU,1.930000e+02,0.000000e+00
GRD,9.788000e+03,0.000000e+00
CYM,5.899200e+04,0.000000e+00
KNA,5.960800e+04,0.000000e+00
URY,1.004749e+10,2.000000e+01


### NETWORKX

In [29]:
nodes = [
    (int(c.Code), {'name': c.Name}) for c in country_codes.itertuples()
]

edges

G = nx.DiGraph()

G.add_nodes_from(nodes)

In [31]:
#G.nodes.data()

### IMPORT/EXPORT TOTAL